In [1]:
import requests
import re
import time
import lxml.html
import openpyxl
import pandas as pd

from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
custom_header = {
    'referer' : 'http://www.riss.kr/index.do',
    #'referer' : 'http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&exQuery=&exQueryText=&order=&onHanja=&strSort=&p_year1=&p_year2=&iStartCount=0&orderBy=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=bib_t&colName=bib_t&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&query=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5',
    'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'  }

# 검색키워드: 질적 연구

In [3]:
#http://www.riss.or.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EC%A7%88%EC%A0%81+%EC%97%B0%EA%B5%AC&exQuery=regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88pyear%3A2021%E2%97%88pyear%3A2020%E2%97%88pyear%3A2019%E2%97%88pyear%3A2018%E2%97%88pyear%3A2017%E2%97%88pyear%3A2016%E2%97%88pyear%3A2015%E2%97%88pyear%3A2014%E2%97%88pyear%3A2013%E2%97%88pyear%3A2012%E2%97%88pyear%3A2011%E2%97%88pyear%3A2010%E2%97%88language%3Akor%E2%97%88kdc%3A5%E2%97%88&exQueryText=%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2021%5D%40%40pyear%3A2021%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2020%5D%40%40pyear%3A2020%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2019%5D%40%40pyear%3A2019%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2018%5D%40%40pyear%3A2018%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2017%5D%40%40pyear%3A2017%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2016%5D%40%40pyear%3A2016%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2015%5D%40%40pyear%3A2015%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2014%5D%40%40pyear%3A2014%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2013%5D%40%40pyear%3A2013%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2012%5D%40%40pyear%3A2012%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2011%5D%40%40pyear%3A2011%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2010%5D%40%40pyear%3A2010%E2%97%88%EC%9E%91%EC%84%B1%EC%96%B8%EC%96%B4+%5B%ED%95%9C%EA%B5%AD%EC%96%B4%5D%40%40language%3Akor%E2%97%88%EC%A3%BC%EC%A0%9C%EB%B6%84%EB%A5%98+%5B%EA%B8%B0%EC%88%A0%EA%B3%BC%ED%95%99%5D%40%40kdc%3A5%E2%97%88&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount=0&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%EC%A7%88%EC%A0%81+%EC%97%B0%EA%B5%AC

In [4]:
paper_link_list = []
#31
for i in range(31):
    before_url = "http://www.riss.or.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EC%A7%88%EC%A0%81+%EC%97%B0%EA%B5%AC&exQuery=regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88pyear%3A2021%E2%97%88pyear%3A2020%E2%97%88pyear%3A2019%E2%97%88pyear%3A2018%E2%97%88pyear%3A2017%E2%97%88pyear%3A2016%E2%97%88pyear%3A2015%E2%97%88pyear%3A2014%E2%97%88pyear%3A2013%E2%97%88pyear%3A2012%E2%97%88pyear%3A2011%E2%97%88pyear%3A2010%E2%97%88language%3Akor%E2%97%88kdc%3A5%E2%97%88&exQueryText=%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2021%5D%40%40pyear%3A2021%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2020%5D%40%40pyear%3A2020%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2019%5D%40%40pyear%3A2019%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2018%5D%40%40pyear%3A2018%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2017%5D%40%40pyear%3A2017%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2016%5D%40%40pyear%3A2016%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2015%5D%40%40pyear%3A2015%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2014%5D%40%40pyear%3A2014%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2013%5D%40%40pyear%3A2013%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2012%5D%40%40pyear%3A2012%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2011%5D%40%40pyear%3A2011%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2010%5D%40%40pyear%3A2010%E2%97%88%EC%9E%91%EC%84%B1%EC%96%B8%EC%96%B4+%5B%ED%95%9C%EA%B5%AD%EC%96%B4%5D%40%40language%3Akor%E2%97%88%EC%A3%BC%EC%A0%9C%EB%B6%84%EB%A5%98+%5B%EA%B8%B0%EC%88%A0%EA%B3%BC%ED%95%99%5D%40%40kdc%3A5%E2%97%88&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount="
    page = i*10
    after_url = "&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%EC%A7%88%EC%A0%81+%EC%97%B0%EA%B5%AC"
    
    url = before_url + str(page) + after_url
    
    page_req = requests.get(url)
    page_html = page_req.text
    page_soup = BeautifulSoup(page_html, 'html.parser')

    #links.append(page_soup.select('#divContent > div.rightContent > div > div.srchResultW > div.srchResultListW > ul > li > div.cont > p.title > a'))
    links = page_soup.select('#divContent > div.rightContent > div > div.srchResultW > div.srchResultListW > ul > li > div.cont > p.title > a')
    
    for link in links:
        ## Tag안의 텍스트
        print(link.text)
        paper_link_list.append(link.get('href'))
    print(len(paper_link_list))

모유수유모의 수유경험에 대한 연구
뇌졸중 노인 환자의 원예요법 체험연구
미술치료 석사 과정생의 미술치료 체험연구 : 호스피스 대상자를 중심으로
산업디자인의 질적 연구 특성 고찰
폭력가정의 아동과 어머니를 위한 집단미술활동에 대한 질적 연구
자녀의 음악치료 경험을 통한 부모의 인식 형성에 관한 질적 연구
약물사용검토 서비스에 대한 개국약사들의 인식을 탐구한 질적 연구
임상간호연구 게재논문 분석: 2009년부터 2015년까지
중년여성의 한국화 미술치료 체험에 관한 현상학적 연구
입원 스트레스에 대한 국내 연구동향 분석 : 대상자 중심으로
10
방과 후 학교에 대한 예술치료사들의 경험연구
정신질환 자녀를 둔 노인의 자아통합과정
노인의 만성질환 경험
손자녀를 돌보는 할머니의 경험 : 내러티브 탐구
임상간호사의 투약오류 경험
성인 지적 장애 자녀 어머니들의 양육 체험
에니어그램 6번 유형의 격정 다스리기
이직한 신입간호사의 학생임상실습경험
간호대학생의 시뮬레이션을 활용한 교육 경험
한국 남성 심근경색증 환자의 질병 체험에 관한 현상학적 연구
20
간호학 전공 남학생의 체험
노인의 낙상 경험
기혼자 간호학생의 학습활동경험
실습지도교수와 간호사의 간호학생 임상실습지도 경험
중환자실 간호사의 임종간호경험 : 시공간성 탐구
재가 여성노인의 불면증 경험
간호사의 심폐소생술의 의미
체중관리 프로그램에 참여하는 중년기 성인의 의사결정 행위
간호사의 전문직업성 가치 : 병원소비자를 대상으로
연구논문 : 패션상품기획자의 정보품질 인식에 대한 질적연구: 이용정보에 대한 만족/불만족요인을 중심으로
30
취학 전 자녀를 둔 어머니의 세대 간 양가감정에 관한 질적연구
한국 작업치료분야 단일대상연구의 질적 수준에 대한 고찰 : 대한작업치료학회지를 중심으로
감각통합치료효과에 대한 단일대상연구의 특성과 질적 수준에 대한 고찰
여성의 물리적·정신적 공간 탐색에 관한 예비연구 (1) : 일본 전문직 여성을 중심으로
외상 경험자를 위한 국내 예술치료 연구 동향분석
서울지역 대학생의 자발적 졸업연기에 대

허약노인의 주관적 건강인식
남자 간호학생의 군 제대 후 복학 시 적응경험
정신건강간호사가 받은 슈퍼비전 경험
저소득 여성노인의 우울 경험
정신보건간호사가 지역사회 정신간호 현장에서 경험하는 윤리적 문제
HIV 감염인 남편과 살아가는 감염되지 않은 여성의 삶의 경험
여가활동으로서의 무도스포츠 역할 재고를 위한 고찰
미술치료 반응성 애착장애 아동의 사회적 행동과 애착행동 증진에 미치는 영향
편측 공간무시가 있는 뇌졸중 환자의 프리즘 적응 치료에 관한 고찰
불안장애 아동의 감각자극에 대한 생리학적 반응: 체계적 고찰
250
국유 자연휴양림의 이용자 행태 및 요구
뇌성마비 유형 중 편마비 아동을 위한 수정된 강제유도 운동치료의 효과에 대한 체계적 고찰
PET-CT의 방사선안전관리 가이드라인 개발 : 행위와 환경을 중심으로
노인의 인지 기능 향상을 위한 전산화 인지 훈련의 효과 : 체계적 고찰
아동의 실행기능 평가도구에 대한 체계적 고찰 : 감각통합기반중심으로
집단미술치료가 이혼가정아동의 자기표현과 사회성향상에 미치는 효과 - 지역아동센터 아동을 중심으로
집단 복합 중재가 초기치매 노인의 인지기능과 우울에 미치는 효과
정신보건 작업치료의 효과에 관한 체계적 고찰
지역사회 노인 운전자를 대상으로 한 운전중재 프로그램 효과에 관한 메타분석
원저 : 안경광학과 전공 교과목에 대한 재학생과 안경사의 요구 비교
260
외상성 뇌손상 운전자의 운전능력과 측정방법에 관한 체계적 고찰
정보영재를 위한 온라인 콘텐츠 평가 준거 개발
강제 유도 운동치료(Constraint-Induced Movement Therapy)가 편마비를 가진 뇌성마비 아동의 상지 기능에 미치는 효과 : 메타분석
강제 유도 운동치료(Constraint-Induced Movement Therapy)가 편마비를 가진 뇌성마비 아동의 상지 기능에 미치는 효과: 메타분석
다문화 이해를 위한 원예활동이 초등학생의 자아존중감, 배려, 다문화 이해 및 태도에 미치는 영향
국가인적자원개발 컨소시엄 훈련인프라 사업의 심사 및 성

In [5]:
paper_link_list

['/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=9cbe14de3e76a71bffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=ec00ce757b3761a0ffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=1988d2262dc01fddffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=5f53e49bc51f1360ffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=6028b3dd22b11d934884a65323211ff0',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=c96c35a90e210fa0ffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=e20f7e67afeb5d7effe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=6d8638bd17e758674884a65323211ff0',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=cacb8dacfb1259e9ffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c

In [6]:
len(paper_link_list)

308

In [7]:
paper_url = []

for paper_link in tqdm(paper_link_list):
    tmp_url = "http://www.riss.kr" + paper_link
    paper_url.append(tmp_url)

100%|████████████████████████████████████████████████████████████████████████████████████████| 308/308 [00:00<?, ?it/s]


In [8]:
paper_url

['http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=9cbe14de3e76a71bffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=ec00ce757b3761a0ffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=1988d2262dc01fddffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=5f53e49bc51f1360ffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=6028b3dd22b11d934884a65323211ff0',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=c96c35a90e210fa0ffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=e20f7e67afeb5d7effe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=6d8638bd17e758674884a65323211ff0',
 'http://www.ris

In [9]:
len(paper_url)

308

In [10]:
write_wb = openpyxl.Workbook()
write_ws = write_wb.active

In [11]:
"""
write_ws = write_wb.active
write_ws['A1'] = 'title_kor'
write_ws['B1'] = 'title_eng'
write_ws['C1'] = 'author'
write_ws['D1'] = 'book'
write_ws['E1'] = 'year'
write_ws['F1'] = 'keyword'
write_ws['G1'] = 'abstract_kor'
write_ws['H1'] = 'abstract_eng'
"""

"\nwrite_ws = write_wb.active\nwrite_ws['A1'] = 'title_kor'\nwrite_ws['B1'] = 'title_eng'\nwrite_ws['C1'] = 'author'\nwrite_ws['D1'] = 'book'\nwrite_ws['E1'] = 'year'\nwrite_ws['F1'] = 'keyword'\nwrite_ws['G1'] = 'abstract_kor'\nwrite_ws['H1'] = 'abstract_eng'\n"

In [12]:
write_ws = write_wb.active
write_ws['A1'] = 'title'
write_ws['B1'] = 'author'
write_ws['C1'] = 'book'
write_ws['D1'] = 'year'
write_ws['E1'] = 'keyword'
write_ws['F1'] = 'abstract_kor'
write_ws['G1'] = 'abstract_eng'

In [13]:
start = time.time()

for i, url in enumerate(paper_url):
    res = requests.get(url, headers = custom_header)
    element = lxml.html.fromstring(res.text)

    try:
        tmp_title = element.cssselect('h3.title')
        tmp_title = tmp_title[0].text_content()
        tmp_title = tmp_title.split("=")
        title_kor = tmp_title[0].strip()
        #title_eng = tmp_title[1].strip()

        tmp_author = element.cssselect('div.infoDetailL > ul > li:nth-child(1) > div > p')
        author = tmp_author[0].text_content().strip()

        tmp_book = element.cssselect('div.infoDetailL > ul > li:nth-child(3) > div > p')
        book = tmp_book[0].text_content().strip()

        tmp_year = element.cssselect('div.infoDetailL > ul > li:nth-child(5) > div > p')
        year = tmp_year[0].text_content().strip()

        tmp_keyword = element.cssselect('div.infoDetailL > ul > li:nth-child(7) > div > p')
        keyword = tmp_keyword[0].text_content().strip()

        tmp_abstract = element.cssselect('div.text.off')
        abstract_kor = tmp_abstract[0].text_content().strip()
        abstract_eng = tmp_abstract[1].text_content().strip()

        print(i, 'paper clear')

    except:
        print(i, 'ERROR')
        continue


    write_ws.append([title_kor, author, book, year, keyword, abstract_kor, abstract_eng])
    write_wb.save('./8_질적_연구.xlsx')
    
print("Running Time : ", time.time() - start)

0 paper clear
1 paper clear
2 paper clear
3 paper clear
4 paper clear
5 paper clear
6 paper clear
7 paper clear
8 paper clear
9 paper clear
10 paper clear
11 paper clear
12 paper clear
13 paper clear
14 paper clear
15 paper clear
16 paper clear
17 paper clear
18 paper clear
19 paper clear
20 paper clear
21 paper clear
22 paper clear
23 paper clear
24 paper clear
25 paper clear
26 paper clear
27 paper clear
28 paper clear
29 ERROR
30 paper clear
31 paper clear
32 paper clear
33 paper clear
34 paper clear
35 paper clear
36 paper clear
37 paper clear
38 paper clear
39 paper clear
40 paper clear
41 paper clear
42 ERROR
43 ERROR
44 ERROR
45 ERROR
46 paper clear
47 paper clear
48 ERROR
49 ERROR
50 ERROR
51 ERROR
52 ERROR
53 ERROR
54 paper clear
55 paper clear
56 ERROR
57 paper clear
58 paper clear
59 ERROR
60 ERROR
61 paper clear
62 ERROR
63 ERROR
64 paper clear
65 paper clear
66 paper clear
67 paper clear
68 paper clear
69 paper clear
70 paper clear
71 paper clear
72 paper clear
73 paper cl